**月報**https://mops.twse.com.tw/mops/web/t21sc04_ifrs

In [1]:
from excellib import set_excel_color_daily
from stocklib import get_stock_price, get_OTC_price, get_otc, get_investor, get_ratio
from stocklib import cal_bar, cal_KD, check_10days, check_10days_otc, get_trust_today
from stocklib import convert_num
# from stock_plot import plot_K_chart


from finance import get_yearly, get_df_monthly, get_df_monthly_check, get_df_quarterly
from finance import get_this_quarter, get_quarter_income, get_quarter_finance, set_growth_eps
from finance import get_df_quarterly_all, get_df_yearly_all

In [2]:
import requests
import shutil
import time
from datetime import date, timedelta, datetime
from dateutil.relativedelta import relativedelta
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import pandas as pd
import numpy as np
import yfinance as yf
import json
import matplotlib.pyplot as plt
import mplfinance as mpf
import seaborn as sns
from tqdm import tqdm
from glob import glob
%matplotlib inline

# ua = UserAgent()
root = '../'
path_trust = root + 'trust/'
path_check = root + 'daily_check/'
path_investor = root + 'investor/'
path_flow = root + 'flow/'

# 取得今天日期

In [3]:
d_today = date.today() #- timedelta(days=1)

today_tw = d_today - relativedelta(years=1911)
this_year = d_today.year
this_quarter = get_this_quarter()
n = 1
if d_today.weekday() == 0:
    n = 3
d_yesterday = d_today - timedelta(days=n)
s_yesterday = (d_today - timedelta(days=n)).strftime("%Y%m%d")
s_today_md = (d_today).strftime("%m/%d")
s_yesterday_md = (d_today - timedelta(days=n)).strftime("%m/%d")
s_today = d_today.strftime("%Y%m%d")

s_today_tw = today_tw.strftime("%Y/%m/%d").lstrip('0')
s_thismonth_tw = today_tw.strftime("%Y/%m").lstrip('0')
print('今天:', d_today, '昨天:', s_yesterday)
# 台灣證券交易所

今天: 2023-04-26 昨天: 20230425


In [4]:
df_yesterday = get_stock_price(d_yesterday)

df_today = get_stock_price(d_today)
df_today.head(2)

取得今日上市股票資訊: 20230425
取得今日上市股票資訊: 20230426


,name,amount,open,high,low,close
id,,,,,,
0050,元大台灣50,14209,116.55,116.60,115.65,116.10
0051,元大中型100,44,56.30,56.45,56.00,56.35


In [5]:
df_investor = get_investor(d_today)

lst_investor = list(df_investor[ (df_investor['investor']>0) & (df_investor['trust']>0)].index)
print('外資, 投信同步買超筆數：', len(lst_investor))
print(list(lst_investor))
df_investor.head(2)

外資, 投信同步買超筆數： 1561
外資, 投信同步買超筆數： 1
['03136P']


,investor,trust
id,,
3231,16493,0
2303,10871,0


In [6]:
lst_investor_trust = list(df_investor[ (df_investor['investor']>100) | (df_investor['trust']>100)].index)
print('外資 or 投信買超筆數：', len(lst_investor_trust))

外資 or 投信買超筆數： 307


# 取得交易價量前30名

In [7]:
df_amt = df_today.copy()
df_amt['value_amt'] = df_amt['amount'] * df_amt['close'] # 成交量 * 收盤價(億)
df_amt['value_amt'] //= 100000
df_amt.sort_values('value_amt', ascending=False, inplace=True)
df_amt = df_amt[df_amt['close'] > df_amt['open']]
lst_amt = list(df_amt.head(30).index)
print('today:', lst_amt)

path_amt30 = './amt30.txt'
with open(path_amt30, 'r') as f:
    lst_amt30_yesterday = f.read().split('\n')
print('yesterday:', lst_amt30_yesterday)

with open(path_amt30, 'w') as f:
    for stock_id in lst_amt:
        f.write(stock_id + "\n")
lst_amt30_new = [i for i in lst_amt if i not in lst_amt30_yesterday]
print('\nnew:', lst_amt30_new)
df_amt = df_amt.head(30)
df_amt_new = df_amt.reset_index()
df_amt_new[df_amt_new['id'].isin(lst_amt30_new)]

today: ['3443', '3661', '3037', '2454', '2317', '2303', '6669', '8046', '3231', '3034', '1503', '6531', '2412', '1905', '2204', '3008', '2471', '1590', '8478', '3017', '1101', '2453', '6282', '1727', '1605', '4919', '1513', '8996', '5269', '6415']
yesterday: ['8222', '00632R', '1721', '6753', '00673R', '00680L', '00671R', '2548', '2912', '5522', '1810', '2385', '3419', '3501', '3563', '9910', '1219', '04284U', '042820', '042314', '04230U', '04283U', '04280U', '04281U', '04229U', '04228U', '04275U', '042765', '042342', '042334', '']

new: ['3443', '3661', '3037', '2454', '2317', '2303', '6669', '8046', '3231', '3034', '1503', '6531', '2412', '1905', '2204', '3008', '2471', '1590', '8478', '3017', '1101', '2453', '6282', '1727', '1605', '4919', '1513', '8996', '5269', '6415']


,id,name,amount,open,high,low,close,value_amt
0,3443,創意,5808,926.00,959.00,925.00,957.00,55.0
1,3661,世芯-KY,4736,1140.00,1180.00,1135.00,1170.00,55.0
2,3037,欣興,37295,133.50,140.50,133.50,139.00,51.0
3,2454,聯發科,7417,641.00,652.00,638.00,649.00,48.0
4,2317,鴻海,34093,103.00,104.50,103.00,103.50,35.0
5,2303,聯電,68224,48.05,48.65,47.80,48.40,33.0
6,6669,緯穎,2949,1105.00,1145.00,1095.00,1135.00,33.0
7,8046,南電,9664,257.00,269.00,256.00,269.00,25.0
8,3231,緯創,57213,43.85,45.00,43.80,45.00,25.0
9,3034,聯詠,6245,405.00,413.00,397.50,412.00,25.0


# 昨天外資、投信同步買超，檢查隔天是否有上漲

In [8]:
import yfinance as yf
def cal_mean_20(stock_id):
    data_yf = yf.Ticker(stock_id+'.TW')
    df = data_yf.history(period="1mo")
    df.sort_index(ascending=False, inplace=True)
    
    price_20 = round(np.mean(df['Close'][:20]),2)
    return price_20

In [9]:
# path_track_yesterday = path_investor + 'investors_2days_{}.xlsx'.format(s_yesterday)
# # shutil.copy2(file_track, file_track.replace('.xlsx', '_backup.xlsx')) 

# df_track_yes = pd.read_excel(path_track_yesterday, index_col=None, engine='openpyxl')
# df_track_yes['id'] = df_track_yes['id'].astype(str)
# lst_track_yes = list(set(df_track_yes['id']))
# print('昨天外資投信同步買超筆數：', len(lst_track_yes), s_yesterday)
# df_track_yes.set_index('id', inplace=True)
# df_track_close_yes = df_track_yes[['close']]
# df_track_close_yes.columns = ['close_yesterday']

# # 昨天外資、投信同步買進，只保留隔天有上漲的股票
# df_track_today = df_today[df_today.index.isin(lst_track_yes)].copy()
# df_track_today['increase'] = 0
# df_track_today['mean20'] = 0
# df_track_today = df_track_today.join(df_track_close_yes)
# df_track_today['increase'] = df_track_today['close'] - df_track_today['close_yesterday']
# df_track_today = df_track_today[df_track_today['increase']>=0]
# df_track_today['mean20'] = df_track_today.index.to_series().apply(cal_mean_20)
# # <月線2%內
# df_track_today = df_track_today[df_track_today['close'] < df_track_today['mean20'] * 1.02]
# lst_increase = list(df_track_today.index)
# print('隔天上漲且 <月線2%內筆數：', len(lst_increase), lst_increase)

# # join (今日股價、今日法人買賣超)
# df_track_today_inv = df_track_today.join(df_investor)
# df_track_today_inv.insert(1, 'date', s_today)
# df_track_today_inv

# # # save file
# df_track_yes = df_track_yes[df_track_yes.index.isin(lst_increase)]
# df_track_updated = pd.concat([df_track_yes.iloc[:,:-1],df_track_today_inv])
# df_track_updated.reset_index(inplace=True)
# df_track_updated.sort_values(['id', 'date'], ascending=[True, False], inplace=True)
# df_track_updated.to_excel(path_track_yesterday, index=False)
# df_track_updated

# 今天外資、投信同步買超

In [10]:
path_track_today = path_investor + 'investors_2days_{}.xlsx'.format(s_today)

df_inv_price = df_today[df_today.index.isin(lst_investor)].copy()
df_inv_price['increase'] = 0
df_inv_price['mean20'] = 0

df_inv_today = df_inv_price.join(df_investor)
df_inv_today

print('今天外資投信同步買超筆數：', len(lst_investor), lst_investor)

# save file
df_inv_today.insert(1, 'date', s_today)
df_inv_today.reset_index(inplace=True)
df_inv_today.to_excel(path_track_today, index=False)
df_inv_today.head(2)

今天外資投信同步買超筆數： 1 ['03136P']


,id,name,date,amount,open,high,low,close,increase,mean20,investor,trust
0,03136P,台積電麥證28售04,20230426,2679,0.66,0.73,0.66,0.7,0,0,2,1911


## 外資、投信同步買超 (檢查交易量突破2倍、收盤價突破10天盤整)

In [11]:
## 今日符合篩選條件，新增至追蹤
# 漲幅 > 1.5%，K棒 > 1%
df_add = df_today.loc[lst_investor,:]
df_add.sort_index(inplace=True)

df_add['rise%'] = round((df_add['close'] - df_yesterday['close'])/df_yesterday['close']*100, 2)  # 漲跌%
df_add['K%'] = round( (df_add['close'] - df_add['open'])/df_add['open']*100, 2 )    # K棒%
df_add = df_add[ (df_add['rise%']>1.5) & (df_add['K%']>1) ]
df_add_min = df_add.copy()
df_add_min_yes = df_add.copy()

df_add = check_10days(df_add)
lst_add_ori = list(df_add.index)
print('今日新增:',len(lst_add_ori),'筆\n', lst_add_ori)

lst_add = df_add.index
df_add = df_add.loc[lst_add,:]
df_add

查無資料 Hist < 10 1
今日新增: 0 筆
 []


,name,amount,open,high,low,close,rise%,K%
id,,,,,,,,


## 量價前30名 (檢查交易量突破2倍、收盤價突破10天盤整)

In [12]:
print('量價前30名:', lst_amt)
df_amt = check_10days(df_amt)
lst_amt = list(df_amt.index)
print('量價前30名，10日盤整:',len(lst_amt),'筆', lst_amt)
df_amt

量價前30名: ['3443', '3661', '3037', '2454', '2317', '2303', '6669', '8046', '3231', '3034', '1503', '6531', '2412', '1905', '2204', '3008', '2471', '1590', '8478', '3017', '1101', '2453', '6282', '1727', '1605', '4919', '1513', '8996', '5269', '6415']
量價前30名，10日盤整: 0 筆 []


,name,amount,open,high,low,close,value_amt,check
id,,,,,,,,


## 外資 or 投信 買超

In [13]:
## 外資 or 投信 買超
df_inv_t = df_today.loc[lst_investor_trust,:]
df_inv_t.sort_index(inplace=True)

df_inv_t['rise%'] = round((df_inv_t['close'] - df_yesterday['close'])/df_yesterday['close']*100, 2)  # 漲跌%
df_inv_t['K%'] = round( (df_inv_t['close'] - df_inv_t['open'])/df_inv_t['open']*100, 2 )    # K棒%
df_inv_t = df_inv_t[ (df_inv_t['rise%']>1) & (df_inv_t['K%']>1) ]
df_inv_min = df_inv_t.copy()
df_inv_min_yes = df_inv_t.copy()

df_inv_t = check_10days(df_inv_t)
lst_inv_t = list(df_inv_t.index)
print('今日新增:',len(lst_inv_t),'筆\n', lst_inv_t)

df_inv_t

031100.TW: No data found, symbol may be delisted
查無資料 Hist < 10 0
查無資料 Hist < 10 1
查無資料 Hist < 10 1
查無資料 Hist < 10 1
查無資料 Hist < 10 1
查無資料 Hist < 10 1
039792.TW: No data found, symbol may be delisted
查無資料 Hist < 10 0
039793.TW: No data found, symbol may be delisted
查無資料 Hist < 10 0
查無資料 Hist < 10 1
查無資料 Hist < 10 1
查無資料 Hist < 10 1
1108, 收盤價：17.0, 10日內最高價：16.4, 布林通道差：3.7, 布林帶寬：0.24
5522, 收盤價：61.3, 10日內最高價：59.1, 布林通道差：3.91, 布林帶寬：0.07
6412, 收盤價：82.5, 10日內最高價：81.8, 布林通道差：6.51, 布林帶寬：0.08
今日新增: 3 筆
 ['1108', '5522', '6412']


,name,amount,open,high,low,close,rise%,K%,check
id,,,,,,,,,
1108,幸福,11103,15.95,17.2,15.95,17.0,6.92,6.58,9.0
5522,遠雄,3212,58.70,61.5,58.60,61.3,4.79,4.43,9.0
6412,群電,1044,78.70,82.7,78.70,82.5,4.96,4.83,9.0


## 外資、投信同步買超 (檢查創新低、昨日最低點)

In [14]:
# 今日創新低
df_add_min = check_10days(df_add_min, check='min')
lst_add_min = list(df_add_min.index)
print('今日新增:',len(lst_add_min),'筆\n', lst_add_min)
df_add_min

查無資料 Hist < 10 1
今日新增: 0 筆
 []


,name,amount,open,high,low,close,rise%,K%
id,,,,,,,,


In [15]:
# 昨日最低點
df_add_min_yes = check_10days(df_add_min_yes, check='yes')
lst_add_min_yes = list(df_add_min_yes.index)
print('今日新增:',len(lst_add_min_yes),'筆\n', lst_add_min_yes)
df_add_min_yes

查無資料 Hist < 10 1
今日新增: 0 筆
 []


,name,amount,open,high,low,close,rise%,K%
id,,,,,,,,


## 外資 or 投信 買超(檢查創新低、昨日最低點)

In [16]:
df_inv_min = check_10days(df_inv_min, check='min')
lst_inv_min = list(df_inv_min.index)
print('今日新增:',len(lst_inv_min),'筆\n', lst_inv_min)

df_inv_min

031100.TW: No data found, symbol may be delisted
查無資料 Hist < 10 0
查無資料 Hist < 10 1
查無資料 Hist < 10 1
查無資料 Hist < 10 1
查無資料 Hist < 10 1
查無資料 Hist < 10 1
039792.TW: No data found, symbol may be delisted
查無資料 Hist < 10 0
039793.TW: No data found, symbol may be delisted
查無資料 Hist < 10 0
查無資料 Hist < 10 1
查無資料 Hist < 10 1
查無資料 Hist < 10 1
今日新增: 0 筆
 []


,name,amount,open,high,low,close,rise%,K%,check
id,,,,,,,,,


In [17]:
df_inv_min_yes = df_inv_t.copy()
df_inv_min_yes = check_10days(df_inv_min_yes, check='yes')
lst_inv_min_yes = list(df_inv_min_yes.index)
print('今日新增:',len(lst_inv_min_yes),'筆\n', lst_inv_min_yes)

df_inv_min_yes

今日新增: 0 筆
 []


,name,amount,open,high,low,close,rise%,K%,check
id,,,,,,,,,


# K、D值

In [18]:
import os, argparse, datetime, logging, traceback, time
import time, random, warnings
import pandas as pd
from tqdm import tqdm
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains
from selenium import webdriver

def main(IdDaysDict, verbose=False):
    ### Output
    col = 'StockID, StockName, Date, KRaw, DRaw, JRaw, K, D, J'
    dfStockKDJ = pd.DataFrame(columns = col.split(', '))

    ### XpathSetting ###
    # XpathSettingButton ='/html/body/div[23]/input'
    # XpathIdxLengthInput = '/html/body/div[21]/table/tbody/tr[3]/td/table[1]/tbody/tr[2]/td[1]/table/tbody/tr/td[1]/table/tbody/tr[4]/td[2]/input'
    # XpathShiftInput = '/html/body/div[21]/table/tbody/tr[3]/td/table[1]/tbody/tr[2]/td[1]/table/tbody/tr/td[1]/table/tbody/tr[10]/td[2]/input'
    # XpathSettingConfirmButton = f'/html/body/div[24]/table/tbody/tr[3]/td/table[2]/tbody/tr[2]/td[1]/button'
    # XpathPreviousDayButton = '/html/body/div[20]/nobr/table/tbody/tr/td[2]/nobr/input[2]'
    XpathAnchor = '/html/body/table[2]/tbody/tr/td[3]/table[2]/tbody/tr[2]/td/canvas'
    XpathStockName = '/html/body/table[2]/tbody/tr/td[3]/table[2]/tbody/tr[1]/td/table/tbody/tr/td[2]/nobr/b'
    # XpathDate = '/html/body/div[3]/table/tbody/tr/td/div[1]/table/tbody/tr/td[7]'                           # OldVersion.~20221122
    # XpathK = '/html/body/div[9]/table/tbody/tr/td[1]/nobr[1]/font'
    # XpathD = '/html/body/div[9]/table/tbody/tr/td[1]/nobr[2]/font'
    # XpathJ = '/html/body/div[9]/table/tbody/tr/td[1]/nobr[3]/font'

    ### webdriver
    if verbose: print('Opening Webdirver...')
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--window-size=1920,1080')
    chrome_options.add_argument('--start-maximized')
    # chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--headless')
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()),
                              options=chrome_options)
    driver.maximize_window()
    FirstSetting = True
    ErrorMessage = []
    progress = IdDaysDict.items() if verbose else tqdm(IdDaysDict.items())
    for STOCK_ID, DAY in progress:
        if verbose:print(f'Fetching  StockID  :  {STOCK_ID}  information ...\t(1/{DAY})\t', end = '')
        url = f'https://goodinfo.tw/tw/ShowK_Chart.asp?STOCK_ID={STOCK_ID}&CHT_CAT2=DATE'
        driver.get(url)
        # OldVersion
            # settingButton = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, 'StockCanvas_btnConfig')))
        try:
            settingButton = driver.find_element(By.ID, 'StockCanvas_btnConfig')
            settingButton.click()
        except Exception as e:
            print(f'There is no data available for this StockID : {STOCK_ID}')
            ErrorMessage.append(e)
            continue

        if FirstSetting:
            idxLengthInput = driver.find_element(By.ID, 'CFG_InitIndexLength')
            shiftInput = driver.find_element(By.ID, 'CFG_CtrlShiftCnt')
            for n in range(20,25):
                try:
                    XpathSettingConfirmButton = f'/html/body/div[{n}]/table/tbody/tr[3]/td/table[2]/tbody/tr[2]/td[1]/button'
                    settingConfirmButton = driver.find_element(By.XPATH, XpathSettingConfirmButton)
                except:
                    pass
            idxLengthInput.send_keys('\b\b\b1')
            shiftInput.send_keys('\b\b1')
            driver.execute_script("arguments[0].scrollIntoView(true);",
                                  driver.find_element(By.XPATH, XpathAnchor))
            settingConfirmButton.click()
        else:
            driver.execute_script("arguments[0].scrollIntoView(true);",
                                  driver.find_element(By.XPATH, XpathAnchor))

        for i in range(DAY):
            stockName = driver.find_element(By.XPATH, XpathStockName).text.split()[1]
            
            for n in range(3,5):
                try:
                    XpathDate = f'/html/body/div[{n}]/table/tbody/tr/td/div[1]/table/tbody/tr/td[7]'
                    date = driver.find_element(By.XPATH, XpathDate).text.strip()
                except:
                    pass
            
            for n in range(9,15):
                try:
                    XpathK = f'/html/body/div[{n}]/table/tbody/tr/td[3]/nobr[1]/font'
                    XpathD = f'/html/body/div[{n}]/table/tbody/tr/td[3]/nobr[2]/font'
                    XpathJ = f'/html/body/div[{n}]/table/tbody/tr/td[3]/nobr[3]/font'
                    KRaw, DRaw, JRaw = map(lambda x: driver.find_element(By.XPATH, x).text[3:], (XpathK, XpathD, XpathJ))
                except:
                    pass

            K, D, J = map(lambda x: float(x[:-1]), (KRaw, DRaw, JRaw))
            dfStockKDJ.loc[len(dfStockKDJ)] = tuple([STOCK_ID, stockName, date, KRaw, DRaw, JRaw, K, D, J])
            if verbose: print(f'|| {STOCK_ID} | {stockName} | {date} | K: {KRaw:^6s} | D: {DRaw:^6s} | J: {JRaw:^6s} ||')
            if i == DAY - 1: break # early break

            # change date
            if verbose: print(f"Fetching the previous day's information ...\t({i+2}/{DAY})\t", end = '')
            previousDayButton = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, 'btnCtrlShiftLeft')))
            previousDayButton.click()
            time.sleep(1)
        if verbose: print()
    driver.close()
    if verbose:
        print('All query is done!!')
        print('There is some Error occured, please check!')
        for e in ErrorMessage:
            print(e)
    return dfStockKDJ


In [19]:
df_track = pd.read_excel('../stock_track.xlsx')
lst_track = lst_inv_t + lst_add_min_yes + lst_add_min
lst_track += list(df_track['stock_id'])
print(len(lst_track))

dic_track = {str(i):1 for i in lst_track}
df_KD = main(dic_track, verbose=False)
display(df_KD)

58


[WDM] - Downloading: 100%|████████████████████████████████████████████████████████| 6.80M/6.80M [00:02<00:00, 2.99MB/s]
 71%|█████████████████████████████████████████████████████████▉                        | 41/58 [02:56<00:45,  2.67s/it]

There is no data available for this StockID : 6235


 72%|███████████████████████████████████████████████████████████▍                      | 42/58 [02:56<00:31,  1.95s/it]

There is no data available for this StockID : 6257


...

 98%|████████████████████████████████████████████████████████████████████████████████▌ | 57/58 [03:00<00:00,  4.03it/s]

There is no data available for this StockID : 9921


100%|██████████████████████████████████████████████████████████████████████████████████| 58/58 [03:00<00:00,  3.12s/it]

There is no data available for this StockID : 9941


,StockID,StockName,Date,KRaw,DRaw,JRaw,K,D,J
0,1108,幸福,2023/04/26 (三),68.2▲,64.7▲,75▲,68.20,64.7,75.00
1,5522,遠雄,2023/04/26 (三),63.4▲,51.7▲,86.8▲,63.40,51.7,86.80
2,6412,群電,2023/04/26 (三),44.2▲,34.6▲,63.5▲,44.20,34.6,63.50
3,1476,儒鴻,2023/04/26 (三),34.2▲,37.7▼,27.1▲,34.20,37.7,27.10
4,1513,中興電,2023/04/26 (三),25.6▼,35.1▼,6.66▲,25.60,35.1,6.66
5,1519,華城,2023/04/26 (三),40.4▼,58.7▼,3.76▼,40.40,58.7,3.76
6,1712,興農,2023/04/26 (三),26.7▲,26.2▲,27.6▲,26.70,26.2,27.60
7,1736,喬山,2023/04/26 (三),26.9▲,30.1▼,20.5▲,26.90,30.1,20.50
8,1909,榮成,2023/04/26 (三),35.2▼,42.5▼,20.6▲,35.20,42.5,20.60
9,2027,大成鋼,2023/04/26 (三),12.5▼,20.1▼,-2.77▲,12.50,20.1,-2.77


In [20]:
df_KD[(df_KD['K']<30) & (df_KD['K']<df_KD['D'])].sort_values('K')

,StockID,StockName,Date,KRaw,DRaw,JRaw,K,D,J
13,2330,台積電,2023/04/26 (三),7.24▼,12.6▼,-3.53▼,7.24,12.6,-3.53
20,2454,聯發科,2023/04/26 (三),7.85▲,9.8▼,3.87▲,7.85,9.8,3.87
38,5388,中磊,2023/04/26 (三),8.31▲,9.8▼,5.41▲,8.31,9.8,5.41
24,2606,裕民,2023/04/26 (三),9.5▼,20.5▼,-12.4▲,9.50,20.5,-12.40
25,2607,榮運,2023/04/26 (三),9.8▼,19.6▼,-9.8▲,9.80,19.6,-9.80
33,3596,智易,2023/04/26 (三),11.4▲,18▼,-1.82▲,11.40,18.0,-1.82
29,3035,智原,2023/04/26 (三),11.8▼,15.8▼,3.57▼,11.80,15.8,3.57
35,3653,健策,2023/04/26 (三),12.4▼,16.2▼,4.77▲,12.40,16.2,4.77
9,2027,大成鋼,2023/04/26 (三),12.5▼,20.1▼,-2.77▲,12.50,20.1,-2.77
22,2481,強茂,2023/04/26 (三),12.7▼,19.5▼,-0.7▲,12.70,19.5,-0.70


# 下載Gmail股票交易excel

In [21]:
from google_apis import create_service

CLIENT_FILE = 'client_secret.json'
API_NAME = 'gmail'
API_VERSION = 'v1'
SCOPES = ['https://mail.google.com/']

service = create_service(CLIENT_FILE, API_NAME, API_VERSION, SCOPES)

def search_emails(query_string, label_ids=None):
    try:
        message_list_response = service.users().messages().list(
                userId='me',
                labelIds=label_ids,
                q=query_string
        ).execute()
        message_items = message_list_response.get('messages')
        next_page_token = message_list_response.get('nextPageToken')
        return message_items
    except Exception as e:
        print(e)
        
def get_file_data(message_id, attachment_id, file_name, save_locatioin):
    response = service.users().messages().attachments().get(
        userId='me',
        messageId=message_id,
        id=attachment_id
    ).execute()
    
    file_data = base64.urlsafe_b64decode(response.get('data').encode('utf-8'))
    return file_data

def get_message_detail(message_id, msg_format='metadata', metadata_headers=None):
    message_detail = service.users().messages().get(
        userId='me',
        id=message_id,
        format=msg_format,
        metadataHeaders=metadata_headers
    ).execute()
    return message_detail

gmail v1 service created successfully


In [22]:
import os
import base64
from tqdm import tqdm
from datetime import datetime

def download_excel():
    query_string = 'has:attachment'
    email_messages = search_emails(query_string)
    save_folder = os.path.join(os.path.dirname(os.getcwd()), 'fugle')
    this_month = s_today[:-2]
    
    last_month = (s_today[:4] + str(datetime.now().month - 1).zfill(2) if datetime.now().month > 1 else f"{datetime.now().year-1}12")
    if not os.path.exists(os.path.join(save_folder, this_month)):
        os.mkdir(os.path.join(save_folder, this_month))
    lst_excel = glob(os.path.join(save_folder, this_month, '交易明細*'))
    if len(lst_excel) > 0:
        latest_file = lst_excel[-1]
    else:
        latest_file = glob(os.path.join(save_folder, last_month, '交易明細*'))[-1]
    latest_file = os.path.basename(latest_file)
    print('local latest file', latest_file)

    for email_message in tqdm(email_messages):
        message_detail = get_message_detail(email_message['id'], msg_format='full', metadata_headers=['parts'])
        if '富果帳戶 交易明細' in message_detail['snippet'] and not 'Forwarded' in message_detail['snippet']:
            message_detail_payload = message_detail.get('payload')
            if 'parts' in message_detail_payload:
                for msg_payload in message_detail_payload['parts']:
                    file_name = msg_payload['filename']
                    body = msg_payload['body']
                    if 'attachmentId' in body:
                        folder = file_name[4:10]
                        if file_name == latest_file:
                            return
                        attachment_id = body['attachmentId']
                        attachment_content = get_file_data(email_message['id'], attachment_id, file_name, save_folder)
                        with open(os.path.join(save_folder, folder, file_name), 'wb') as f:
                            f.write(attachment_content)
                            print(f"File {file_name} is saved at {save_folder} {folder}")
download_excel()

local latest file 交易明細20230425.xlsx


  1%|▊                                                                                 | 1/100 [00:00<00:56,  1.75it/s]

File 交易明細20230426.xlsx is saved at D:\Gitlab\stock\fugle 202304


  1%|▊                                                                                 | 1/100 [00:00<01:24,  1.18it/s]


In [23]:
df_profit = pd.DataFrame()
from glob import glob
save_folder = os.path.join(os.path.dirname(os.getcwd()), 'fugle')
this_month = s_today[:-2]
for filename in glob(os.path.join(save_folder, '202211', '交易明細*')):
    df_profit = pd.concat([df_profit, pd.read_excel(filename)])
df_profit

D:\envs\stock\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
D:\envs\stock\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
D:\envs\stock\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
D:\envs\stock\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
D:\envs\stock\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Wor

,交易別,代碼,商品名稱,成交股數,成交單價,成交價金,手續費,交易稅,融資自備款,融資金額,融券擔保品,融券保證金,融券費,淨收付,交易日期
0,現買,2607,榮運,2000,26.75,53500,76,0,NaN,NaN,NaN,NaN,0,-53576,20221101
1,現買,6670,復盛應用,200,185.00,37000,52,0,NaN,NaN,NaN,NaN,0,-37052,20221101
0,現買,2345,智邦,150,241.00,36150,51,0,NaN,NaN,NaN,NaN,0,-36201,20221102
1,現買,6670,復盛應用,150,187.50,28124,40,0,NaN,NaN,NaN,NaN,0,-28164,20221102
2,現買,9802,鈺齊-KY,300,141.50,42450,60,0,NaN,NaN,NaN,NaN,0,-42510,20221102
0,現賣,8478,東哥遊艇,200,242.00,48400,68,145,NaN,NaN,NaN,NaN,0,48187,20221103
1,現買,9802,鈺齊-KY,100,139.50,13950,19,0,NaN,NaN,NaN,NaN,0,-13969,20221103
2,現買,9941,裕融,200,138.00,27600,39,0,NaN,NaN,NaN,NaN,0,-27639,20221103
0,現賣,2345,智邦,500,246.00,123000,175,369,NaN,NaN,NaN,NaN,0,122456,20221110
1,現賣,9802,鈺齊-KY,500,136.00,68000,96,204,NaN,NaN,NaN,NaN,0,67700,20221110


In [24]:
df_profit['成交股數'] = df_profit.apply(lambda row: row['成交股數']*(-1) if row['交易別']=='現賣' else row['成交股數'], axis=1)
df_profit[df_profit['交易別'].isin(['現買','現賣'])].groupby(['代碼','商品名稱']).sum()

,,交易別,成交股數,成交單價,成交價金,手續費,交易稅,融資自備款,融資金額,融券擔保品,融券保證金,融券費,淨收付,交易日期
代碼,商品名稱,,,,,,,,,,,,,
2059,川湖,現賣現賣現買現買,-176,1654.50,176512,249,376,0.0,0.0,0.0,0.0,0,73695,80884462
2345,智邦,現買現賣,-350,487.00,159150,226,369,0.0,0.0,0.0,0.0,0,86255,40442212
2457,飛宏,現買現賣,0,79.75,159500,226,254,0.0,0.0,0.0,0.0,0,9820,40442243
2607,榮運,現買現賣,0,54.00,108000,153,163,0.0,0.0,0.0,0.0,0,684,40442219
6285,啟碁,現買,600,82.30,49380,70,0,0.0,0.0,0.0,0.0,0,-49450,20221117
6670,復盛應用,現買現買,350,372.50,65124,92,0,0.0,0.0,0.0,0.0,0,-65216,40442203
8478,東哥遊艇,現賣,-200,242.00,48400,68,145,0.0,0.0,0.0,0.0,0,48187,20221103
9802,鈺齊-KY,現買現買現賣現買,200,569.50,170149,240,204,0.0,0.0,0.0,0.0,0,-34593,80884431
9941,裕融,現買,200,138.00,27600,39,0,0.0,0.0,0.0,0.0,0,-27639,20221103


# 上市上櫃 各類股 資金流向
**找出資金流向幅度增長最高的類股**

In [25]:
# 上市、上櫃資金流向表
from fake_useragent import UserAgent
url_flow = 'https://fubon-ebrokerdj.fbs.com.tw/z/zb/zba/zba.djhtm'
ua = UserAgent()
agent = ua.random
headers = {'user-agent':agent}
r = requests.get(url_flow, headers=headers)

# tables = pd.read_html(url_flow)
tables = pd.read_html(r.text)
df = tables[-1]
df.head(3)

,0,1,2,3,4,5,6,7
0,上市資金流向表 最後更新時間: 04/26,上市資金流向表 最後更新時間: 04/26,上市資金流向表 最後更新時間: 04/26,上市資金流向表 最後更新時間: 04/26,上市資金流向表 最後更新時間: 04/26,上市資金流向表 最後更新時間: 04/26,上市資金流向表 最後更新時間: 04/26,上市資金流向表 最後更新時間: 04/26
1,類股名稱,流向率,類股名稱,流向率,類股名稱,流向率,類股名稱,流向率
2,水泥,0.84%,食品,0.63%,塑膠,1.08%,紡織纖維,0.47%


In [26]:
def get_stock_type_format(df):
    df_all = pd.DataFrame()
    for i in range(1, len(df.columns)//2 + 1):
        i_end = i * 2
        df_tmp = df.iloc[:, i_end-2:i_end]
        df_all = pd.concat([df_all, df_tmp])
    df_all.columns = ['type_name', 'flow_rate']
    df_all.dropna(inplace=True)
    df_all.reset_index(drop=True, inplace=True)
    df_all['flow_rate'] = df_all['flow_rate'].apply(lambda x: x.replace('%', ''))
    df_all['flow_rate'] = df_all['flow_rate'].astype(float)
    df_all = df_all.sort_values('flow_rate', ascending=False)
    
    df_all_t = df_all.T
    df_all_t.columns = df_all_t.iloc[0,:]
    df_all_t = df_all_t.iloc[1:,:]
    df_all_t.insert(0, 'date', value=s_today)
#     df_all_t.to_excel('stock_type.xlsx', index=False)
    return df_all_t

def get_flow_diff(df_new, df_old):
    df_new = df_new.iloc[:, 1:]
    df_new.reset_index(inplace=True, drop=True)
    
    df_yes = df_old.iloc[[-1], :]
    df_yes = df_yes.iloc[:, 1:]
    df_yes.reset_index(inplace=True, drop=True)
    
    df_diff = df_new.subtract(df_yes, fill_value=0)
    df_diff.insert(0, 'date', 'Difference')
    return df_diff

def cal_type_flow(path_file, df_new):
    print(path_file)
    df = pd.read_excel(path_flow + path_file, index_col=None, engine='openpyxl')
    df_ori = df.iloc[:-1, :]
    df_diff = get_flow_diff(df_new, df_ori)
    df_type = pd.concat([df_ori, df_new])
    df_type = pd.concat([df_type, df_diff])
    df_type.to_excel(path_flow + path_file, index=False)
    return df_type.reset_index(drop=True)

In [27]:
df_type = df.iloc[2:10, :]
df_type.columns = df.iloc[1,:]

df_type_new = get_stock_type_format(df_type)

cal_type_flow('stock_type.xlsx', df_type_new).iloc[-3:,:]

stock_type.xlsx


,date,電子,半導體,航運業,電子零組件,化學生技醫療,金融保險,化工,電腦及週邊設備,光電,...,紡織纖維,電子通路,食品,水泥,觀光,橡膠,油電燃氣,造紙,資訊服務,玻璃陶瓷
126,20230425,58.82,31.29,6.12,10.45,4.26,2.07,1.49,6.46,3.04,...,0.29,0.41,0.83,0.9,1.95,0.16,0.1,1.17,1.22,0.13
127,20230426,60.37,31.35,5.45,10.75,4.13,2.7,2.21,6.98,2.9,...,0.47,0.45,0.63,0.84,1.58,0.14,0.16,1.47,1.6,0.09
128,Difference,1.55,0.06,-0.67,0.3,-0.13,0.63,0.72,0.52,-0.14,...,0.18,0.04,-0.2,-0.06,-0.37,-0.02,0.06,0.3,0.38,-0.04


In [28]:
df_otc = df.iloc[12:17, :]
df_otc.columns = df.iloc[11,:]

df_otc_new = get_stock_type_format(df_otc)

cal_type_flow('stock_type_otc.xlsx', df_otc_new).iloc[-3:, :]

stock_type_otc.xlsx


,date,電子,半導體,電子零組件,生技醫療,光電,通信網路,其他電子,電腦及週邊設備,電機機械,化工,文化創意,其他,觀光,資訊服務,鋼鐵,電子通路,航運業,建材營造,紡織纖維
126,20230425,63.7,26.41,7.79,23.18,6.24,6.71,4.17,5.34,3.6,0.57,1.38,1.93,0.85,6.41,2.06,0.62,0.12,0.14,0.05
127,20230426,70.52,27.99,5.78,18.56,6.23,5.58,3.46,4.19,3.73,0.44,1.43,1.61,0.44,16.81,1.45,0.47,0.11,0.12,0.03
128,Difference,6.82,1.58,-2.01,-4.62,-0.01,-1.13,-0.71,-1.15,0.13,-0.13,0.05,-0.32,-0.41,10.4,-0.61,-0.15,-0.01,-0.02,-0.02


# 畫日K線圖、週K線圖，儲存每天股票資料

In [29]:
file_name = root + 'stock_20210825_1203.json'
with open(file_name) as f:
    dic_stock = json.load(f)

latest_date_file = datetime.strptime(sorted(dic_stock)[-1], '%Y-%m-%d').date()
print('目前記錄日期至:',latest_date_file, '新增前:', len(dic_stock), '筆')

目前記錄日期至: 2023-04-25 新增前: 342 筆


In [30]:

import time
import requests
from io import StringIO
import pandas as pd
import numpy as np
import datetime

def crawl_price(date):
    r = requests.post('http://www.twse.com.tw/exchangeReport/MI_INDEX?response=csv&date=' + str(date).split(' ')[0].replace('-','') + '&type=ALL')
    print(str(date).split(' ')[0].replace('-',''))
    ret = pd.read_csv(StringIO("\n".join([i.translate({ord(c): None for c in ' '}) 
                                        for i in r.text.split('\n') 
                                        if len(i.split('",')) == 17 and i[0] != '='])), header=0)
    ret = ret.set_index('證券代號')
    ret['成交金額'] = ret['成交金額'].str.replace(',','')
    ret['成交股數'] = ret['成交股數'].str.replace(',','')
    return ret


data_stock = {}
date_today = d_today
n_days = (d_today - latest_date_file).days
fail_count = 0
allow_continuous_fail_count = 5
print('須補天數:', n_days)
while len(data_stock) < n_days:
    print('parsing', date_today)
    # 使用 crawPrice 爬資料
    try:
        # 抓資料
        if date_today.weekday()  in [5,6]:
            print('Weekend!')
            date_today -= datetime.timedelta(days=1)
            continue
        data_stock[date_today] = crawl_price(date_today)
        if not str(date_today) in dic_stock.keys():
            dic_stock[str(date_today)] = data_stock[date_today].to_dict()
            print("add to dict:", str(date_today))
        print('success!')
        fail_count = 0
    except:
        # 假日爬不到
        print('fail! check the date is holiday')
        fail_count += 1
        if fail_count == allow_continuous_fail_count:
            raise
        break
    
    # 減一天
    date_today -= datetime.timedelta(days=1)
    time.sleep(10)

print('新增後:', len(dic_stock))
with open(file_name, 'w') as f:
    json.dump(dic_stock, f)

須補天數: 1
parsing 2023-04-26
20230426
add to dict: 2023-04-26
success!
新增後: 343


# K、D值

In [31]:
# def plot_daily_weekly_K(lst_stock_id, dic_stock):
#     df_close = pd.DataFrame({k:d['收盤價'] for k,d in dic_stock.items()}).transpose()
#     df_close.index = pd.to_datetime(df_close.index)

#     df_open = pd.DataFrame({k:d['開盤價'] for k,d in dic_stock.items()}).transpose()
#     df_open.index = pd.to_datetime(df_open.index)

#     df_high = pd.DataFrame({k:d['最高價'] for k,d in dic_stock.items()}).transpose()
#     df_high.index = pd.to_datetime(df_high.index)

#     df_low = pd.DataFrame({k:d['最低價'] for k,d in dic_stock.items()}).transpose()
#     df_low.index = pd.to_datetime(df_low.index)

#     df_volume = pd.DataFrame({k:d['成交股數'] for k,d in dic_stock.items()}).transpose()
#     df_volume.index = pd.to_datetime(df_volume.index)
    
#     # =========================== 畫K線圖 =================================================
#     for stock_id in tqdm(lst_stock_id):
#         thisyear = '2022'
#         print(stock_id)
#         if not stock_id in df_close.keys():
#             print(stock_id, '查無資料')
#             continue
#     #     stock_id = '2603'
#         dic_a_stock = {
#             'close':df_close[stock_id][thisyear].dropna().apply(lambda x:x.replace(',','')).astype(float),
#             'open':df_open[stock_id][thisyear].dropna().apply(lambda x:x.replace(',','')).astype(float),
#             'high':df_high[stock_id][thisyear].dropna().apply(lambda x:x.replace(',','')).astype(float),
#             'low':df_low[stock_id][thisyear].dropna().apply(lambda x:x.replace(',','')).astype(float),
#             'volume': df_volume[stock_id][thisyear].dropna().apply(lambda x:x.replace(',','')).astype(float),
#         }

#         # Daily DataFrame
#         df_stock = pd.DataFrame(dic_a_stock)
#         df_stock.insert(0,'stock_id', stock_id)
#         df_stock.sort_index(inplace=True)

#         fig_daily = plot_K_chart(df_stock, 'daily')

#         # Weekly DataFrame
#         period_type = '1W'
#         df_week = df_stock.resample(period_type).last()
#         df_week['close'] = df_stock['close'].resample(period_type).last()
#         df_week['open'] = df_stock['open'].resample(period_type).first()
#         df_week['high'] = df_stock['high'].resample(period_type).max()
#         df_week['low'] = df_stock['low'].resample(period_type).min()
#         df_week['volume'] = df_stock['volume'].resample(period_type).sum()

#         fig_weekly = plot_K_chart(df_week, 'weekly')

In [32]:
# df_close = pd.DataFrame({k:d['收盤價'] for k,d in dic_stock.items()}).transpose()
# df_close.index = pd.to_datetime(df_close.index)

# df_open = pd.DataFrame({k:d['開盤價'] for k,d in dic_stock.items()}).transpose()
# df_open.index = pd.to_datetime(df_open.index)

# df_high = pd.DataFrame({k:d['最高價'] for k,d in dic_stock.items()}).transpose()
# df_high.index = pd.to_datetime(df_high.index)

# df_low = pd.DataFrame({k:d['最低價'] for k,d in dic_stock.items()}).transpose()
# df_low.index = pd.to_datetime(df_low.index)

# df_volume = pd.DataFrame({k:d['成交股數'] for k,d in dic_stock.items()}).transpose()
# df_volume.index = pd.to_datetime(df_volume.index)

# # =========================== 畫K線圖 =================================================
# stock_id = '2603'
# thisyear = '2022'
# print(stock_id)
# if not stock_id in df_close.keys():
#     print(stock_id, '查無資料')
# #     stock_id = '2603'
# dic_a_stock = {
#     'close':df_close[stock_id][thisyear].dropna().apply(lambda x:x.replace(',','')).astype(float),
#     'open':df_open[stock_id][thisyear].dropna().apply(lambda x:x.replace(',','')).astype(float),
#     'high':df_high[stock_id][thisyear].dropna().apply(lambda x:x.replace(',','')).astype(float),
#     'low':df_low[stock_id][thisyear].dropna().apply(lambda x:x.replace(',','')).astype(float),
#     'volume': df_volume[stock_id][thisyear].dropna().apply(lambda x:x.replace(',','')).astype(float),
# }

# # Daily DataFrame
# df_stock = pd.DataFrame(dic_a_stock)
# df_stock.insert(0,'stock_id', stock_id)
# df_stock.sort_index(inplace=True)
# df_stock

In [33]:
# print('今日新增 && 在量價前30名', list(set(lst_add_ori)& set(lst_amt)) )
# print('昨天外資投信同步買超筆數 & 今天上漲：', len(lst_increase), '筆')
# plot_daily_weekly_K(lst_increase, dic_stock)

In [34]:
# print('今日觀察：', len(lst_add_ori), '筆')
# plot_daily_weekly_K(lst_add_ori, dic_stock)

In [35]:
# print('量價前30名：', len(lst_amt),'筆\n', lst_amt)
# lst_amt_without_add = [i for i in lst_amt if i not in lst_add_ori]
# plot_daily_weekly_K(lst_amt_without_add, dic_stock)

# 預測EPS

In [36]:
# def get_EPS(lst_stock_id, filename):
#     df_yearly_all = get_df_yearly_all() # 寫入歷年EPS、本益比，估算今年股價min, max範圍
#     df_quarterly_all = get_df_quarterly_all()
#     df_estimate = pd.DataFrame(columns = list(df_quarterly_all.columns)+['普通股數'])

#     # 取得公司發行股數
#     path_company_stock_num = root + 'company_list.xlsx'
#     df_company_stock_num = pd.read_excel(path_company_stock_num, engine='openpyxl', dtype={'stock_id':str})
#     df_company_stock_num.set_index('stock_id', inplace=True)
    
#     lst_eps = []
#     for i in range(len(lst_stock_id)):
#         stock_id = lst_stock_id[i]
#         t_wait = np.random.randint(5,15)
#         print(stock_id, ', wait ', t_wait)
#         df = get_df_quarterly(stock_id)

#         if df is None:
#             print('df is None')
#             continue
#         if df.loc[df.index[0], '毛利率(%)'] < 20:
#             print(stock_id, '：毛利率', df.loc[df.index[0], '毛利率(%)'], '< 20%，跳過')
#             continue
#         elif df.loc[df.index[0], '營益率(%)'] < 8:
#             print(stock_id, '：營益率' , df.loc[df.index[0], '營益率(%)'] , ' < 8%，跳過')
#             continue

#         lst_eps.append(stock_id)
#         # 計算歷年同季的營收成長率、EPS
#         df = set_growth_eps(df, df_today, stock_id)

#         # 將年度EPS寫到歷年excel裡
#         lst_year = set(df.index.to_series().apply(lambda x:x[:4]))
#         min_year = int(min(lst_year)) # 若最小年是2018 ，只能確定2019都有資料
#         lst_year = [y for y in lst_year if int(y) > min_year]
#         lst_EPS = [{'year':y, 'eps': df.loc[str(y)+'Q1', 'EPS(Y)']} for y in lst_year]

#         df.reset_index(inplace=True)
#         df = df.rename(columns={'index':'quarter'}) # reset_index後column名稱是index

#         stock_name = ""
#         if stock_id in df_today.index:
#             stock_name = df_today.loc[stock_id, 'name']
#             df_yearly = get_yearly(stock_id, lst_EPS, df['quarter'][0][:4], stock_name)    
#             df_yearly_all = df_yearly_all.append(df_yearly, ignore_index=True)

#         df.insert(0, 'name', value=stock_name)
#         df.insert(0, 'stock_id', value=stock_id)

#         df_quarterly_all = df_quarterly_all.append(df, ignore_index=True)
#         time.sleep(t_wait)

#         # ===============================
#         # 預測EPS ( check_EPS )
#         # ===============================
#         df_new_quarter = get_quarter_finance(stock_id, df, this_year, this_quarter, df_company_stock_num, df_today)
#         df_concat = pd.concat([df, df_new_quarter])
#         df_concat.sort_values('quarter', ascending=False, inplace=True)
#         df_estimate = df_estimate.append(df_concat)

#     #預測EPS ( check_EPS )
#     df_estimate.to_excel(path_check+'estimate_{}_{}.xlsx'.format(filename, s_today), index=False)
#     return lst_eps